In [ ]:
import pandas as pad 
import numpy as np
from numpy.core.numeric import NaN

import datetime

import matplotlib.pyplot as plt
import seaborn as sns

import statistics
from scipy import stats
from scipy.stats import pearsonr
from scipy.stats import ttest_ind


import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp as multi 

from sklearn.preprocessing import StandardScaler, OneHotEncoder



from joblib import dump, load

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, GridSearchCV

from sklearn.preprocessing import *
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

from sklearn.metrics import *
from sklearn.model_selection import KFold
from sklearn.metrics import  make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import r2_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import make_classification
from sklearn.metrics import roc_curve

In [ ]:
df           = pad.read_excel('DATA_IA_CQ_Halcyon.xlsx')
list_to_drop = ['ID Patient', 'Nom du Case', 'G_2.5/2.5', 'G_3/3', 'G_2/3', 'G_2/2.5', 'G_3/2', 'ID faisceau ']
df_drop      = df.drop(list_to_drop, axis =1)
df_drop

In [ ]:
to_predict_features = ['Class_2.5/2.5', 'Class_3/3', 'Class_3/2', 'Class_2/3', 'Class_2/2.5']

Y = df_drop[to_predict_features]
X = df_drop.drop(to_predict_features, axis = 1)

#### There is not interest to keep other metrics than SAS10 BA BI as statistical anaylysis have shown

In [ ]:
Y = df_drop['Class_2/2.5']
X = df_drop[['SAS10', 'BA', 'BI']]

In [ ]:
X_np   = np.array(X)
sc     = StandardScaler()
X_norm = sc.fit_transform(X_np)
X      = X_norm

In [ ]:
dump(sc, 'StandardScaler_SAS10_BA_BI.joblib')
dump(X, 'X_norm_SAS10_BA_BI.joblib')
dump(Y, 'Y_2-25.joblib')

In [ ]:
X = load('X_norm_SAS10_BA_BI.joblib')
Y = load('Y.joblib')

In [ ]:
X = load('X_norm.joblib')
Y = load('Y.joblib')

## Entrainement des modèles

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,Y['Class_2/2.5'], test_size = 0.20, random_state = 10)

#ros = RandomOverSampler(sampling_strategy = 'minority')
#X_train_res, y_train_res = ros.fit_resample(X_train, y_train)

In [ ]:
# Création des modèles
List_of_models = [LinearDiscriminantAnalysis(), RidgeClassifier(), KNeighborsClassifier(), GaussianNB(), DecisionTreeClassifier(), SVC(), SGDClassifier(), RandomForestClassifier()]
List_of_models_for_graph = ["LinearDiscriminant", "Ridge", "KNeighbors", "GaussianNB", "DecisionTree", "SVC", "SGD", "RandomForestClassifier"]

def run_model_and_performance_check(model):

    # Choix du modèle et entraînement du modèle
    model_class = model
    model_class.fit(X_train, y_train)

    # Prédiction du modèle et archivage des résultats
    y_pred = model_class.predict(X_test)

    # Création des dataframes résultats
    results_classification = np.array([model_class.score(X_train,y_train), model_class.score(X_test,y_test)])
    df_results = pad.DataFrame(index = ["Score entrainement", "Score de prédiction"], columns = [str(model)[:-2]])
    df_results[str(model)[:-2]] = results_classification

    return df_results

df_results = pad.DataFrame(index = ["Score entrainement", "Score de prédiction", "MAE", "RMSE", "median absolute error"], columns = ["LinearDiscriminant"])

for i in range(len(List_of_models)):
  model_class = List_of_models[i] 
  model_class.fit(X_train, y_train)
  y_pred = model_class.predict(X_test)
  results_classification = np.array([model_class.score(X_train,y_train), model_class.score(X_test,y_test), mean_absolute_error(y_test,y_pred), np.sqrt(mean_squared_error(y_test,y_pred)), median_absolute_error(y_test,y_pred)])
  df_results[List_of_models_for_graph[i]] = results_classification

custom_palette = [sns.xkcd_rgb["windows blue"], sns.xkcd_rgb["pale red"], sns.xkcd_rgb["medium green"], "orange", "blue","yellow", "purple", "deeppink", "brown", "teal", "black"] 
sns.set_palette(custom_palette)

df_graph = df_results.transpose()
fig, axs = plt.subplots(1, 2, figsize=(12, 6))
ax1 = sns.barplot(x=df_graph.index, y=df_graph["Score entrainement"].values, data=df_graph, ax=axs[0], palette = custom_palette)
ax2 = sns.barplot(x=df_graph.index, y=df_graph["Score de prédiction"].values, data=df_graph, ax=axs[1], palette = custom_palette)
ax1.set_xticklabels(ax1.get_xticklabels(),rotation=50)
ax1.set_title('Efficacité des modèles pour la prédiction du G_2.5/2.5')
ax1.set_ylabel('Score entrainement')
ax1.set(ylim=(0, 1))
ax2.set_xticklabels(ax2.get_xticklabels(),rotation=50)
ax2.set_title('Efficacité des modèles pour la prédiction du G_2.5/2.5')
ax2.set_ylabel('Score de prédiction')
ax2.set(ylim=(0, 1))

plt.tight_layout()
#fig.savefig("Performance modèles classification all data", dpi=400)

In [ ]:
# Modèle LinearDiscriminant 
LinearDiscriminant_parameters = {'solver' : ['svd', 'lsqr', 'eigen'], 
                                 'store_covariance' : [True, False],
                                 'tol' : [0.0001,0.0002,0.0003]}

LinearDiscriminant_GridSearchCV = GridSearchCV(estimator = LinearDiscriminantAnalysis(), param_grid = LinearDiscriminant_parameters, cv = 5, n_jobs=-1)
LinearDiscriminant_GridSearchCV.fit(X_train, y_train)
LinearDiscriminant_GridSearchCV.best_params_
print("LinearDiscriminant best param = " + str(LinearDiscriminant_GridSearchCV.best_params_))

In [ ]:
# Modèle RidgeClassifier
RidgeClassifier_parameters = {'alpha' : list(range(1,20)),
                              'fit_intercept' : [True, False],
                              'copy_X' : [True, False],
                              'tol' : [0.0001,0.0002,0.0003],
                              'solver' : ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']}

RidgeClassifier_GridSearchCV = GridSearchCV(estimator = RidgeClassifier(), param_grid = RidgeClassifier_parameters, cv = 5, n_jobs=-1)
RidgeClassifier_GridSearchCV.fit(X_train, y_train)
RidgeClassifier_GridSearchCV.best_params_
print("RidgeClassifier best param = " + str(RidgeClassifier_GridSearchCV.best_params_))

In [ ]:
# Modèle KNeighbors
KNeighborsClassifier_parameters = {#'n_neighbors' : list(range(1,50)),
                                   #'leaf_size' : list(range(1,30)), 
                                   'p':[1,2], 
                                   'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'], 
                                   'metric' : ['minkowski','euclidean','manhattan']}

KNeighborsClassifier_GridSearchCV = GridSearchCV(estimator = KNeighborsClassifier(), param_grid = KNeighborsClassifier_parameters, cv = 5, n_jobs=-1)
KNeighborsClassifier_GridSearchCV.fit(X_train, y_train)
KNeighborsClassifier_GridSearchCV.best_params_
print("KNeighborsClassifier best param = " + str(KNeighborsClassifier_GridSearchCV.best_params_))

In [ ]:
# Modèle GaussianNB
GaussianNB_parameters = {'var_smoothing': np.logspace(0,-9, num=100)}
GaussianNB_GridSearchCV = GridSearchCV(estimator = GaussianNB(), param_grid = GaussianNB_parameters, cv = 5, n_jobs=-1)
GaussianNB_GridSearchCV.fit(X_train, y_train)
GaussianNB_GridSearchCV.best_params_

print("GaussianNB best param = " +str(GaussianNB_GridSearchCV.best_params_))

In [ ]:
# Modèle DecisionTree
DecisionTreeClassifier_parameters = {#'max_features' : ['auto', 'sqrt', '“log2'],
                                     'max_depth': [2, 10, 15,18,20],
                                     'min_samples_leaf': [1, 2, 5, 10, 20, 50, 100],
                                     "min_samples_split": [2, 6, 20],
                                     'criterion': ["gini", "entropy"],
                                     'splitter' : ['best', 'random'],
                                     'min_samples_split' : np.linspace(0.1, 1.0, 5, endpoint=True).tolist()}

DecisionTreeClassifier_GridSearchCV = GridSearchCV(estimator = DecisionTreeClassifier(), param_grid = DecisionTreeClassifier_parameters, cv = 5, n_jobs=-1, verbose = 2)
DecisionTreeClassifier_GridSearchCV.fit(X_train, y_train)
DecisionTreeClassifier_GridSearchCV.best_params_
print("DecisionTreeClassifier best param = " +str(DecisionTreeClassifier_GridSearchCV.best_params_))

In [ ]:
# Modèle SVC
SVC_parameters = {#'C': [1, 10, 50, 100, 200, 300],
                  'gamma': [0.0001, 0.001, 0.01, 0.1, 1],
                  'kernel': ['rbf', 'linear', 'poly', 'sigmoid'],
                  'probability': [True, False]}
                  
SVC_GridSearchCV = GridSearchCV(estimator = SVC(), param_grid = SVC_parameters, cv=5, n_jobs=-1, verbose=2)
SVC_GridSearchCV.fit(X_train, y_train)
SVC_GridSearchCV.best_params_
print("SVC best param = " +str(SVC_GridSearchCV.best_params_))

In [ ]:
#Modèle SGD
SGD_parameters = {'loss' : ['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron', 'squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
                  'penalty' : ['l1', 'l2', 'elasticnet'], 
                  'alpha' : [1e-5, 1e-4, 1e-3, 1e-2, 1e-1], 
                  'l1_ratio' : [0, 0.10, 0.15, 0.20, 0.30], 
                  'fit_intercept' : [True, False],
                  'tol' : [0.0001,0.0002,0.0003]}


SGD_GridSearchCV = GridSearchCV(estimator = SGDClassifier(), param_grid = SGD_parameters, cv=5, n_jobs=-1, verbose=2)
SGD_GridSearchCV.fit(X_train, y_train)
SGD_GridSearchCV.best_params_
print("SGD best param = " +str(SGD_GridSearchCV.best_params_))


In [ ]:
# Modèle RandomForestClassifier
RandomForestClassifier_parameters = {'criterion' : ['gini', 'entropy', 'log_loss'],
                                     'n_estimators' : [int(x) for x in np.linspace(start = 200, stop = 5000, num = 10)], 
                                     'min_samples_split' : [2,5,7,9,10, 15, 20], 
                                     'min_samples_leaf' : [1,2,5,7,8, 12, 15], 
                                     'max_features' : ['auto', 'sqrt', 'log2'], 
                                     'max_depth' : [int(x) for x in np.linspace(10,110, num = 11), None]}

RandomForestClassifier_GridSearchCV = GridSearchCV(estimator = RandomForestClassifier(), param_grid = RandomForestClassifier_parameters, cv=5, n_jobs=-1, verbose=2, n_iter = 500)
RandomForestClassifier_GridSearchCV.fit(X_train, y_train)
RandomForestClassifier_GridSearchCV.best_params_
coefficients = RandomForestClassifier_GridSearchCV.best_estimator_.feature_importances_
print("RandomForestClassifier best param = " +str(RandomForestClassifier_GridSearchCV.best_params_))

In [ ]:
# Modèle RandomForestClassifier
RandomForestClassifier_parameters = {'criterion' : ['gini', 'entropy'],
                                     'n_estimators' : [1,10,20,30, 100, 200, 400], 
                                     'min_samples_split' : [2,5,7,9,10], 
                                     'min_samples_leaf' : [1,2,5,7,9,10], 
                                     'max_features' : ['auto', 'sqrt', 'log2']}


RandomForestClassifier_GridSearchCV = GridSearchCV(estimator = RandomForestClassifier(), param_grid = RandomForestClassifier_parameters, cv=5, n_jobs=-1, verbose=2)
RandomForestClassifier_GridSearchCV.fit(X_train, y_train)
RandomForestClassifier_GridSearchCV.best_params_
coefficients = RandomForestClassifier_GridSearchCV.best_estimator_.feature_importances_
print("RandomForestClassifier best param = " +str(RandomForestClassifier_GridSearchCV.best_params_))

In [ ]:
print(LinearDiscriminant_GridSearchCV.best_params_)

print(RidgeClassifier_GridSearchCV.best_params_)

print(KNeighborsClassifier_GridSearchCV.best_params_)

print(GaussianNB_GridSearchCV.best_params_)

print(DecisionTreeClassifier_GridSearchCV.best_params_)

print(SVC_GridSearchCV.best_params_)

print(SGD_GridSearchCV.best_params_)

print(RandomForestClassifier_GridSearchCV.best_params_)

In [ ]:
List_of_models = [LinearDiscriminantAnalysis(solver = 'svd', store_covariance = True, tol = 0.0001),
                  RidgeClassifier(alpha = 13, copy_X = True, fit_intercept = False, solver = 'auto', tol = 0.0001),
                  KNeighborsClassifier(algorithm = 'auto', metric = 'minkowski', p = 1),
                  GaussianNB(var_smoothing = 0.2848035868435802),
                  DecisionTreeClassifier(criterion = 'gini', max_depth = 2, min_samples_leaf = 100, min_samples_split = 0.1, splitter = 'best'),
                  SVC(gamma = 0.1, kernel = 'sigmoid', probability = True),
                  SGDClassifier(alpha = 0.001, fit_intercept = False, l1_ratio = 0.2, loss = 'huber', penalty = 'l2', tol = 0.0002),
                  RandomForestClassifier(criterion = 'entropy', max_features = 'auto', min_samples_leaf = 2, min_samples_split = 5, n_estimators = 30)]

List_of_models_for_graph = ["LinearDiscriminant", "Ridge", "KNeighbors", "GaussianNB", "DecisionTree", "SVC", "SGD", "RandomForestClassifier"]

df_results = pad.DataFrame(index = ["Score entrainement", "Score de prédiction", "MAE", "RMSE", "median absolute error"], columns = ["LinearDiscriminant"])

custom_palette = [sns.xkcd_rgb["windows blue"], sns.xkcd_rgb["pale red"], sns.xkcd_rgb["green blue"], "orange", sns.xkcd_rgb["blue"],sns.xkcd_rgb["sunny yellow"], sns.xkcd_rgb["warm purple"], sns.xkcd_rgb["medium green"], "brown", "teal", "black"] 
sns.set_palette(custom_palette)

#custom_palette = sns.color_palette("tab10")

for i in range(len(List_of_models)):
  model_class = List_of_models[i] 
  model_class.fit(X_train, y_train)
  y_pred = model_class.predict(X_test)
  results_classification = np.array([model_class.score(X_train,y_train), model_class.score(X_test,y_test), mean_absolute_error(y_test,y_pred), np.sqrt(mean_squared_error(y_test,y_pred)), median_absolute_error(y_test,y_pred)])
  df_results[List_of_models_for_graph[i]] = results_classification

  df_graph = df_results.transpose()
fig, axs = plt.subplots(1, 2, figsize=(12, 6))
ax1 = sns.barplot(x=df_graph.index, y=df_graph["Score entrainement"].values, data=df_graph, ax=axs[0], palette = custom_palette)
ax2 = sns.barplot(x=df_graph.index, y=df_graph["Score de prédiction"].values, data=df_graph, ax=axs[1], palette = custom_palette)
ax1.set_xticklabels(ax1.get_xticklabels(),rotation=45)
#ax1.set_title('Efficacité des modèles pour la prédiction du gamma moyen \n pour toutes les localisations')
ax1.set_ylabel('Training score')
ax1.set(ylim=(0, 1))
ax2.set_xticklabels(ax2.get_xticklabels(),rotation=45)
#ax2.set_title('Efficacité des modèles pour la prédiction du gamma moyen \n pour toutes les localisations')
ax2.set_ylabel('Validation score')
ax2.set(ylim=(0, 1))

plt.tight_layout()
fig.savefig("Performance modèles ML trois metriques", dpi=400)

In [ ]:
df_results

In [ ]:
#RFC = RandomForestClassifier(criterion = 'gini', max_depth = 80, max_features = 'auto', min_samples_leaf = 1, min_samples_split = 7, n_estimators = 200)

RFC = RandomForestClassifier(criterion = 'entropy', max_features = 'auto', min_samples_leaf = 2, min_samples_split = 5, n_estimators = 30)

In [ ]:
RFC.fit(X_train, y_train)

In [ ]:
pred = RFC.predict(X)

In [ ]:
Y = Y['Class_2/2.5']

In [ ]:
auc = roc_auc_score(Y, pred)
print("AUC = " + str(round(auc,4)))

In [ ]:
y = Y
#y = y[:,1]
yhat = RFC.predict(X)
#yhat = yhat[:,1]

from numpy import argmax
from numpy import sqrt

#yhat = model_rfc.predict_proba(X)
# keep probabilities for the positive outcome only
#yhat = yhat[:, 1]

# calculate roc curves
fpr, tpr, thresholds = roc_curve(y, yhat)
# calculate the g-mean for each threshold
gmeans = sqrt(tpr * (1-fpr))
# locate the index of the largest g-mean
ix = argmax(gmeans)
print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))
auc = roc_auc_score(y, yhat)
print('AUC=%f' %auc)
# plot the roc curve for the model
plt.plot([0,1], [0,1], linestyle='--', label='No Skill')
plt.plot(fpr, tpr, marker='.', label='RandomForestClassifier')
plt.scatter(fpr[ix], tpr[ix], marker='o', color='black', label='Best')
plt.text(0.5,0.8,'AUC=%f' %round(auc,4) ,horizontalalignment='center',
     verticalalignment='center', fontsize=10, color='black')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
# show the plot
plt.tight_layout()

plt.savefig("RFC one class", dpi=400)
plt.show()

In [ ]:
import tensorflow as tf
import seaborn as sns
#pred = model.predict(inputs[test])
conf_matrix_SGD_01 = tf.math.confusion_matrix(labels=Y, predictions=pred)
 
ax_sgd_01 = sns.heatmap(conf_matrix_SGD_01, annot = True, fmt='d')
ax_sgd_01.set_title('RandomForestClassifier, AUC = ' + str(round(auc,4)))
ax_sgd_01.set_ylabel('Réel')
ax_sgd_01.set_xlabel('Prédiction')
plt.savefig("Matrice de confusion RFC", dpi=400)
plt.show()

In [ ]:
TP = 120
TN = 155
FP = 20
FN = 23
sensitivity = TP/(TP+FN)
specificity = TN/(TN+FP)
print("sensitivity = " + str(sensitivity))
print("specificity = " + str(specificity))